In [1]:
import tensorflow 
from tensorflow.keras.layers import Reshape
from keras.layers import Input, Embedding, Reshape, LSTM, Dense, Flatten
from keras.models import Model
import os
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

In [2]:
batch_size = 16
max_sequence_length = 384
max_word_length = 20
embedding_size = 128
lstm_units = 256
num_classes = 14

In [7]:
def process_text_files(directory):
    all_texts = []
    all_texts_extend = []
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                content = re.sub(r'<start_".*?">', '', content)
                content = re.sub(r'<end_".*?">', '', content)
                txt_data = content.split()
                all_texts.append(txt_data)
                all_texts_extend.extend(txt_data)
    
    tokenizer = Tokenizer(char_level=True, lower=False)
    tokenizer.fit_on_texts(all_texts_extend)
    
    all_sequences=[]
    for i in range(len(all_texts)):
        all_sequences.append(tokenizer.texts_to_sequences(all_texts[i]))
        
    max_word_length = 20
    trunc_type = 'post'
    padding_type = 'post'
    
    # Pad each word sequence to the length of the longest word
    all_padded_sequences=[]
    for i in range(len(all_sequences)):
        padded_sequence = pad_sequences(all_sequences[i], maxlen=max_word_length, padding=padding_type, truncating=trunc_type)
        all_padded_sequences.append(padded_sequence)
    
    return all_padded_sequences, tokenizer.word_index

#directory_path = '.'
directory_path = '.'
padded_sequences, word_index = process_text_files(directory_path)

print("-----------------")
print(word_index)
print("-----------------")

padded_arrays = []
for arr in padded_sequences:
    pad_width = ((0, max_sequence_length - len(arr)), (0, 0))
    padded_arr = np.pad(arr, pad_width, mode='constant', constant_values=0)
    padded_arrays.append(padded_arr)


np.set_printoptions(threshold=np.inf)

#son array'i numpy array'ine çevir
x_train=np.array(padded_arrays)

print(x_train.shape)

np.set_printoptions(threshold=1000)

-----------------
{'0': 1, 'A': 2, '1': 3, 'E': 4, 'T': 5, 'I': 6, '2': 7, 'R': 8, 'N': 9, 'L': 10, 'K': 11, '*': 12, 'S': 13, 'O': 14, 'M': 15, '5': 16, ':': 17, '.': 18, '3': 19, '8': 20, '4': 21, '9': 22, 'D': 23, '6': 24, 'U': 25, 'İ': 26, '7': 27, ',': 28, 'B': 29, 'C': 30, 'Y': 31, 'P': 32, 'V': 33, 'e': 34, 'a': 35, 'i': 36, 'H': 37, 'G': 38, 'Z': 39, 'r': 40, 'F': 41, '/': 42, 'Ş': 43, 'n': 44, 'o': 45, 'l': 46, '%': 47, 't': 48, 's': 49, 'Ü': 50, '-': 51, 'k': 52, 'd': 53, 'm': 54, 'u': 55, 'X': 56, 'y': 57, 'c': 58, 'g': 59, 'h': 60, 'z': 61, '#': 62, 'ı': 63, 'b': 64, 'w': 65, 'Ç': 66, 'Ğ': 67, 'Ö': 68, 'p': 69, 'v': 70, 'ş': 71, 'W': 72, 'ü': 73, ')': 74, '(': 75, 'J': 76, 'f': 77, 'x': 78, "'": 79, '+': 80, '$': 81, 'ğ': 82, '»': 83, 'ç': 84, '!': 85, 'Q': 86, '=': 87, 'ö': 88, 'Í': 89, 'ж': 90, '&': 91, 'j': 92, '>': 93, '"': 94, '\\': 95, 'Т': 96, 'О': 97, 'Р': 98, 'q': 99, '×': 100, '@': 101, ';': 102, 'Ø': 103, 'К': 104, '|': 105, '<': 106, 'Á': 107, 'Ú': 108, 'Ș': 109

In [8]:
global wordsList,wordsIndextList

wordsList = []
wordsIndextList = []
max_sequence_length = 384


label_list = ["<start_\"company_name\">",
              "<end_\"company_name\">",
                  "<start_\"date\">",
                  "<end_\"date\">",
                  "<start_\"time\">",
                  "<end_\"time\">",
                  "<start_\"receipt_number\">",
                  "<end_\"receipt_number\">",
                  "<start_\"tax\">",
                  "<end_\"tax\">",
                  "<start_\"amount\">",
                  "<end_\"amount\">"]

label_list2= ["Pad","Others","B_Comp","l_Comp","B_Date","l_Date","B_Time","l_Time",
                  "B_Receipt","l_Receipt","B_Tax","l_Tax","B_Amount","l_Amount"]

 

def one_hot_encode_index_list():
    oneHotEncodedList = []
    unique_labels_len = len(sorted(set(label_list2)))
    for i in wordsIndextList:
        bitlist = list(0 for i in range(unique_labels_len))
        bitlist[i] = 1
        oneHotEncodedList.append(bitlist)

    for i in  range(max_sequence_length-len(oneHotEncodedList)):
        oneHotEncodedList.append([0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    return oneHotEncodedList


def splitfunction(text:str):
    global wordsList,wordsIndextList
    wordsList = []
    wordsIndextList = []
    words = text.split()
    for word in words:
        wordsList.append(word)
        wordsIndextList.append(1)



def indexAssignment(tag,last):
    if(tag==None):
        return None
    index = label_list.index(tag)+2
    if(last == "l"):
       index +=1
    return index


def checkTag(word:str):
    for tag in label_list:
        index = word.find(tag)
        if index != -1:
            return tag    
    return None



def tagingWords():
    control = None
    counter = 0
    for word in wordsList:
        tag = checkTag(word)
        if tag != None:
            if(word.find("<start_") != -1):
                control = tag
            if(word.find("<start_") != -1 and word.find("<end_") == -1):
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            elif(word.find("<start_") != -1 and word.find("<end_") != -1):
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            else:
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            if(word.find("<end_") != -1):
                control = None
        elif control !=None:
            wordsIndextList[counter] = indexAssignment(control,"l")
            counter+=1
        else:
            counter+=1  


def oneHotEncodedFunction(texts):    
    splitfunction(texts)
    tagingWords()
    return np.array(one_hot_encode_index_list())



folder_path = "."
file_list = os.listdir(folder_path)
y_train= []

for file_name in file_list:
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            result = oneHotEncodedFunction(text)
            
            if len(y_train) == 0:
                y_train = np.array([result])
            else:
                y_train = np.concatenate((y_train, [result]), axis=0)
                
y_train.shape

(1094, 384, 14)

In [48]:
# LSTM Modeli

input_shape = (max_sequence_length, max_word_length)
input_layer = Input(shape=(max_sequence_length,max_word_length), batch_size=16)
print(input_layer.shape)


(16, 384, 20)


In [49]:
char_embedding = Embedding(input_dim=max_word_length, output_dim=embedding_size)(input_layer)
print(char_embedding.shape)


(16, 384, 20, 128)


In [50]:
reshaped_embedding = tensorflow.reshape(char_embedding, (batch_size * max_sequence_length, max_word_length, embedding_size))
print(reshaped_embedding.shape)

(6144, 20, 128)


In [51]:
char_lstm, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(reshaped_embedding)
print(char_lstm)

char_lstm2, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(char_lstm, initial_state=[state_h, state_c])
print(char_lstm2)

KerasTensor(type_spec=TensorSpec(shape=(6144, 20, 256), dtype=tf.float32, name=None), name='lstm_6/PartitionedCall:1', description="created by layer 'lstm_6'")
KerasTensor(type_spec=TensorSpec(shape=(6144, 20, 256), dtype=tf.float32, name=None), name='lstm_7/PartitionedCall:1', description="created by layer 'lstm_7'")


In [52]:
reshaped = tensorflow.reshape(state_h, (batch_size,max_sequence_length, state_h.shape[-1]))
print(reshaped.shape)

(16, 384, 256)


In [53]:
word_lstm1, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(reshaped)
print(word_lstm1)
word_lstm2, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(word_lstm1, initial_state=[state_h, state_c])
print(word_lstm2)

KerasTensor(type_spec=TensorSpec(shape=(16, 384, 256), dtype=tf.float32, name=None), name='lstm_8/PartitionedCall:1', description="created by layer 'lstm_8'")
KerasTensor(type_spec=TensorSpec(shape=(16, 384, 256), dtype=tf.float32, name=None), name='lstm_9/PartitionedCall:1', description="created by layer 'lstm_9'")


In [54]:
classification_output = Dense(units=num_classes, activation='softmax')(word_lstm2)
print(word_lstm2)

KerasTensor(type_spec=TensorSpec(shape=(16, 384, 256), dtype=tf.float32, name=None), name='lstm_9/PartitionedCall:1', description="created by layer 'lstm_9'")


In [55]:
model = Model(inputs=input_layer, outputs=[classification_output, char_lstm])


In [56]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(16, 384, 20)]      0           []                               
                                                                                                  
 embedding_2 (Embedding)        (16, 384, 20, 128)   2560        ['input_4[0][0]']                
                                                                                                  
 tf.reshape_5 (TFOpLambda)      (6144, 20, 128)      0           ['embedding_2[0][0]']            
                                                                                                  
 lstm_6 (LSTM)                  [(6144, 20, 256),    394240      ['tf.reshape_5[0][0]']           
                                 (6144, 256),                                               